In [3]:
from affinewarp import PiecewiseWarping, SpikeData
import numpy as np
from affinewarp.visualization import rasters    
import matplotlib.pyplot as plt
from DataPreprocessing import *

In [4]:
mid_brain_circuits=['SCs','SCm','MRN','APN','PAG','ZI']
frontal_circuits=['MOs','PL','ILA','ORB','MOp','SSp']
all_data_path='/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/unzipped_files'

In [19]:
#Load data

data_path= '/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/unzipped_files/Richards_2017-10-31.tar'

trials_intervals = np.load(data_path+'/'+'trials.intervals.npy') # in seconds
spike_times = np.load(data_path+'/'+'spikes.times.npy') * 1000 # Unbinned spike times in ms
trials_gocue_times = np.load(data_path+'/'+'trials.goCue_times.npy') 
trials_response_choice = np.load(data_path+'/'+'trials.response_choice.npy') # -1 left, 1, right, 0 no response
spontaneous_intervals = np.load(data_path+'/'+'spontaneous.intervals.npy')
trials_response_time = np.load(data_path+'/'+'trials.response_times.npy')
spike_clusters = np.load(data_path+'/'+'spikes.clusters.npy')
site_positions = np.load(data_path+'/'+'channels.sitePositions.npy')
clusters_depths = np.load(data_path+'/'+'clusters.depths.npy')
clusters_annotation = np.load(data_path+'/'+'clusters._phy_annotation.npy')
channel_sites = np.load(data_path+'/'+'channels.site.npy')
channels_brainlocation = pd.read_csv(data_path+'/'+'channels.brainLocation.tsv', sep='\t')
clusters_probes = np.load(data_path+'/'+'clusters.probes.npy')
channels_probe = np.load(data_path+'/'+'channels.probe.npy')
trials_visual_time = np.load(data_path+'/'+'trials.visualStim_times.npy')

good_cells, brain_regions,br = get_good_cells(data_path) #
#print(good_cells)

def get_neurons(spike_times,spike_clusters):
    spike_time_cells = np.empty(len(np.unique(spike_clusters)), dtype=object) # Initalise empty object
    for i in range(len(np.unique(spike_clusters))):
      # Create a spike time arrays, where each array in the array is a spike time of a cell
      spike_time_cells[i] = spike_times[(np.where(spike_clusters == i)[0])]
    return spike_time_cells

def make_arrays(spike_time_cells,trials_intervals):
    spikes=[]
    neurons=[]
    trials=[]
    trials_orig=trials_intervals*1000  
    neurons_orig=range(len(spike_time_cells))
    
    for neuron in neurons_orig:
        #spk_ids=spike_time_cells[neuron]
        spk_tms_one_neuron=spike_time_cells[neuron]
        for trial in range(0,260):
            trial_range= np.bitwise_and(spk_tms_one_neuron>=trials_orig[trial][0],spk_tms_one_neuron<=trials_orig[trial][1])
            if trial==0:
                #trial_range= np.bitwise_and(spk_tms_one_neuron>=trials[trial][0],spk_tms_one_neuron<=trials[trial][1])
                subset=spk_tms_one_neuron[trial_range]
            else:
                subset=spk_tms_one_neuron[trial_range]-trials_orig[trial-1][1]
            for spike in subset:
                trials.append(trial)
                spikes.append(spike)
                neurons.append(neuron)
    return neurons,trials,spikes
        
    
def preprocess_for_affine_warp(bin_size):
    good_cells, brain_regions,br = get_good_cells(data_path) # Get brain regions
    spike_time_cells=get_neurons(spike_times,spike_clusters)
    neurons,trials,spikes=make_arrays(spike_time_cells,trials_intervals)
    #neuron_trials=extract_trial_aligned(spike_time_cells,trials_intervals)
    #mean_lst=bin_and_average(neuron_trials,trials_intervals,bin_size=10)
    #dat_for_affine_warp= preprocess_for_affine_warp(neuron_trials,trials_intervals,bin_size=10)
    #spike_time_binned, spike_time_cells, cell_spikes_max = bin_spikes(spike_times,spike_clusters,clusters_annotation,bin_size=bs)
    return neurons,trials,spikes
neurons,trials,spikes=preprocess_for_affine_warp(10)


In [23]:
print(len(spikes))

1553247
